# Counting velos with Julia

A [notebook](https://opendata.utou.ch/presentations/pyzurich%202014.9/) was presented a few years back by [@loleg](http://github.com/loleg) at a [PyZürich meetup](http://www.meetup.com/pyzurich/events/192324292/), serving a brief introduction to loading and processing open data with the [Pandas](http://pandas.pydata.org/) open source data analysis library. The second part of this notebook dealt with visualizing government data from bike counters *(Daten der permanenten Velozählstellen)*. The analysis is heavily inspired by prior work by [Dr. Ralph Straumann](http://geo.ebp.ch/2014/02/18/blick-in-die-werkzeugkiste-offene-daten-in-r-teil1/) (see also [part 2](http://geo.ebp.ch/2014/04/04/blick-in-die-werkzeugkiste-offene-daten-in-r-teil-2/), [part 3](http://geo.ebp.ch/2014/04/25/blick-in-die-werkzeugkiste-offene-daten-in-r-teil-3/)).

Since mobility is a popular subject in Switzerland, and the data source continues to be a source of interest I have rewritten the example using the [Julia language](http://julialang.org/). To run this notebook interactively, install Julia locally and follow the instructions at [IJulia.jl](https://github.com/JuliaLang/IJulia.jl) to set up a notebook server. Or use [JuliaBox](http://www.juliabox.com) to run this code in the cloud.

So let's get started and pull in some useful libraries.

In [1]:
include("../src/TableSchema.jl")
using TableSchema
using Requests
import Requests: get

The City of Zürich has an [open data portal](http://data.stadt-zuerich.ch) that puts together data from all kinds of departments and makes it more accessible to the general public. Let's use this to learn something about this lovely city. Do you like riding your bike through town - or maybe you would like to know just where all those cyclists are rushing off to in the morning? 

Download the latest data with [Requests.jl](https://github.com/JuliaWeb/Requests.jl), and take a peek inside.

In [2]:
verkehr_filename = "2018verkehrszaehlungenwertefussgaengervelo.csv"
verkehr_url = "https://data.stadt-zuerich.ch/dataset/verkehrszaehlungen_werte_fussgaenger_velo/resource/13af0d7d-41e6-4212-aea3-cd04a4646665/download/"
verkehr_url = verkehr_url * verkehr_filename
if !(isfile(verkehr_filename))
    verkehr_request = get(verkehr_url)
    save(verkehr_request, verkehr_filename)
end
t = Table(verkehr_filename)
t.headers

7-element Array{String,1}:
 "\ufeff\"fk_zaehler\""
 "datum"               
 "velo_in"             
 "velo_out"            
 "fuss_in"             
 "fuss_out"            
 "objectid"            

Now let's infer the schema, and create a Data Package.

In [5]:
tr = TableSchema.read(t)
s = Schema()
TableSchema.infer(s, tr, t.headers)
[ (f.name, f.typed) for f in s.fields ]

7-element Array{Tuple{String,String},1}:
 ("\ufeff\"fk_zaehler\"", "string")
 ("datum", "string")               
 ("velo_in", "integer")            
 ("velo_out", "integer")           
 ("fuss_in", "integer")            
 ("fuss_out", "integer")           
 ("objectid", "integer")           